<a href="https://colab.research.google.com/github/Nakiyah24/ExplainableAI_Assignments/blob/main/Assignment8/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Chunk 1: Imports, device, and load ImageNet class names
import os
import random
import math
import requests
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from torchvision.transforms import functional as TF
from PIL import Image
import numpy as np

# Device check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device in use:", device)
if device.type == "cuda":
    print("CUDA available, GPU count:", torch.cuda.device_count())
else:
    print("Running on CPU (this will be slower).")


Device in use: cuda
CUDA available, GPU count: 1


In [3]:
imagenet_url = "https://raw.githubusercontent.com/Nakiyah24/ExplainableAI_Assignments/refs/heads/main/Assignment8/DemoCode/imagenet_classes.txt"
imagenet_txt = "imagenet_classes.txt"

# Download once if missing
if not os.path.exists(imagenet_txt):
    r = requests.get(imagenet_url)
    with open(imagenet_txt, "wb") as f:
        f.write(r.content)
    print("Downloaded imagenet_classes.txt from GitHub.")

# Load and verify
with open(imagenet_txt, "r") as f:
    imagenet_classes = [line.strip() for line in f.readlines() if line.strip()]

target_label = "coffee mug"
target_idx = imagenet_classes.index(target_label)
print(f"✅ Loaded {len(imagenet_classes)} classes.")
print(f"'{target_label}' found at index {target_idx}")

Downloaded imagenet_classes.txt from GitHub.
✅ Loaded 1000 classes.
'coffee mug' found at index 504
